In [5]:
"""Evaluation metric for Santa 2023."""

import pandas as pd
from ast import literal_eval
from dataclasses import dataclass
from sympy.combinatorics import Permutation
from typing import Dict, List


class ParticipantVisibleError(Exception):
    pass


def score(
        solution: pd.DataFrame,
        submission: pd.DataFrame,
        series_id_column_name: str,
        moves_column_name: str,
        puzzle_info_path: str,
        specific_ids: List[str] = None,
) -> float:
    """Santa 2023 evaluation metric.

    Parameters
    ----------
    solution : pd.DataFrame

    submission : pd.DataFrame

    series_id_column_name : str

    moves_column_name : str

    Returns
    -------
    total_num_moves : int
    """
    if list(submission.columns) != [series_id_column_name, moves_column_name]:
        raise ParticipantVisibleError(
            f"Submission must have columns {series_id_column_name} and {moves_column_name}."
        )

    puzzle_info = pd.read_csv(puzzle_info_path, index_col='puzzle_type')
    print(puzzle_info)
    total_num_moves = 0
    counter = 0
    for sol, sub in zip(solution.itertuples(), submission.itertuples()):
        puzzle_id = getattr(sol, series_id_column_name)
        assert puzzle_id == getattr(sub, series_id_column_name)
        allowed_moves = literal_eval(puzzle_info.loc[sol.puzzle_type, 'allowed_moves'])
        allowed_moves = {k: Permutation(v) for k, v in allowed_moves.items()}
        if specific_ids is not None and puzzle_id not in specific_ids:
            continue
        puzzle = Puzzle(
            puzzle_id=puzzle_id,
            allowed_moves=allowed_moves,
            solution_state=sol.solution_state.split(';'),
            initial_state=sol.initial_state.split(';'),
            num_wildcards=sol.num_wildcards,
        )

        # Score submission row
        total_num_moves += score_puzzle(puzzle_id, puzzle, getattr(sub, moves_column_name))
        # print(sol, total_num_moves)

    return total_num_moves


@dataclass
class Puzzle:
    """A permutation puzzle."""

    puzzle_id: str
    allowed_moves: Dict[str, List[int]]
    solution_state: List[str]
    initial_state: List[str]
    num_wildcards: int


def score_puzzle(puzzle_id, puzzle, sub_solution):
    """Score the solution to a permutation puzzle."""
    # Apply submitted sequence of moves to the initial state, from left to right
    moves = sub_solution.split('.')
    state = puzzle.initial_state
    for m in moves:
        power = 1
        if m[0] == "-":
            m = m[1:]
            power = -1
        try:
            p = puzzle.allowed_moves[m]
        except KeyError:
            raise ParticipantVisibleError(f"{m} is not an allowed move for {puzzle_id}.")
        state = (p ** power)(state)

    # Check that submitted moves solve puzzle
    num_wrong_facelets = sum(not(s == t) for s, t in zip(puzzle.solution_state, state))
    if num_wrong_facelets > puzzle.num_wildcards:
        print(puzzle_id, num_wrong_facelets, puzzle.num_wildcards)
        print("init", ";".join(puzzle.initial_state))
        print("sol", ";".join(puzzle.solution_state))
        print("actual", ";".join(state))
        raise ParticipantVisibleError(f"Submitted moves do not solve {puzzle_id}.")

    # The score for this instance is the total number of moves needed to solve the puzzle
    return len(moves)

In [6]:
solution_df = pd.read_csv("./puzzles.csv")
submission_df = pd.read_csv("./submission_new.csv")
submission_df

,id,moves
0,0,r1.-f1
1,1,f0.r1.f1.-d0.-d0.-f0.-r0.f0.d0
2,2,-d1.-r0.f0.-r1.f1.d1.-r1.-f0.d1.f0.d1.d1
3,3,-f0.d0.-r0.f0.-d0.-r0.d0.-f0.-r0.-f0
4,4,-r1.-f0.d0.r0.-d1.-d1.r1.d1.f0.r1.-d1.-r1
...,...,...
393,393,f10.f43.f55.-f43.-f10.-f55.-r3.-f55.-r0.f55.r0...
394,394,f41.r0.f41.f63.f44.r3.-f51.-r3.-f63.f41.-r0.f4...
395,395,-f65.f34.r3.-f54.-r3.-f34.f52.f50.-f57.-r3.-f5...
396,396,-r4.f0.-r3.-f0.f21.f42.-r5.-f42.-f21.-r3.f42.r...


In [3]:
score(solution_df, submission_df, "id", "moves", "./puzzle_info.csv")


                                                    allowed_moves
puzzle_type                                                      
cube_2/2/2      {'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11,...
cube_3/3/3      {'f0': [0, 1, 2, 3, 4, 5, 44, 41, 38, 15, 12, ...
cube_4/4/4      {'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...
cube_5/5/5      {'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...
cube_6/6/6      {'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...
cube_7/7/7      {'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...
cube_8/8/8      {'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...
cube_9/9/9      {'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...
cube_10/10/10   {'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...
cube_19/19/19   {'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...
cube_33/33/33   {'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...
wreath_6/6      {'l': [1, 2, 3, 4, 5, 0, 6, 7, 8, 9], 'r': [6,...
wreath_7/7      {'l': [1, 2, 3, 4, 5, 6, 0, 7, 8, 9, 10, 11], ...
wreath_12/

KeyboardInterrupt: 

In [7]:
import pandas as pd
puzzles = pd.read_csv("./puzzles.csv")
sol = pd.read_csv("./submission_new.csv")
type_to_score = {}
for i in range(len(puzzles)):
    row = puzzles.iloc[i]
    sol_elm = sol.iloc[i]
    if row["puzzle_type"] not in type_to_score:
        type_to_score[row["puzzle_type"]] = [0,0, []]
    type_to_score[row["puzzle_type"]][0] += sol_elm["moves"].count(".") + 1
    type_to_score[row["puzzle_type"]][1] += 1
    type_to_score[row["puzzle_type"]][2].append(i)
# print puzzle types, and for each puzzle type, print the number of puzzles of that type and the total number of moves
# sort type to score
for k, v in sorted(type_to_score.items(), key=lambda item: item[1]):
    print(k, "\t", v[:2])

print(type_to_score["cube_33/33/33"][2])
# globe_2/6 	 [1508, 5]
# globe_6/4 	 [3696, 5]
# wreath_21/21 	 [3836, 5]
# wreath_33/33 	 [12626, 3]
# globe_3/4 	 [5384, 15]
# cube_6/6/6 	 [9512, 12]



wreath_7/7 	 [128, 15]
wreath_6/6 	 [150, 20]
wreath_12/12 	 [173, 10]
cube_2/2/2 	 [315, 30]
globe_2/6 	 [1288, 5]
wreath_21/21 	 [1365, 5]
globe_1/8 	 [2079, 10]
globe_1/16 	 [2241, 5]
globe_6/4 	 [2603, 5]
cube_7/7/7 	 [2841, 5]
cube_3/3/3 	 [2975, 120]
wreath_33/33 	 [4033, 3]
globe_3/4 	 [4494, 15]
cube_8/8/8 	 [4910, 5]
cube_9/9/9 	 [6812, 5]
cube_4/4/4 	 [8426, 60]
cube_6/6/6 	 [9512, 12]
globe_6/8 	 [9796, 5]
cube_10/10/10 	 [10387, 5]
cube_5/5/5 	 [10556, 35]
globe_6/10 	 [13771, 5]
wreath_100/100 	 [14616, 1]
globe_8/25 	 [37315, 2]
globe_3/33 	 [50619, 4]
globe_33/3 	 [58337, 4]
cube_19/19/19 	 [60651, 4]
cube_33/33/33 	 [331530, 3]
[281, 282, 283]


In [ ]:
from sympy.combinatorics import Permutation

sol_df = pd.read_csv("./submission_schnack.csv")
puzzle_info = pd.read_csv("./puzzle_info.csv")
puzzles = pd.read_csv("./puzzles.csv")


def get_target_perm_for_id(id):
    sol_string_for_id = sol_df.iloc[id]["moves"]
    puzzle_type = puzzles.iloc[id]["puzzle_type"]
    allowed_move_str = puzzle_info[puzzle_info["puzzle_type"] == puzzle_type]["allowed_moves"].iloc[0]
    move_name_to_perm = eval(allowed_move_str)
    # map all elements to their permutation
    move_name_to_perm = {k: Permutation(v) for k, v in move_name_to_perm.items()}
    print(move_name_to_perm)
    def apply_move_string_to_perm(move_string):
        moves = move_string.split(".")
        result = Permutation()
        for move in moves:
            if move[0] == "-":
                result = move_name_to_perm[move[1:]]**-1 * result
            else:
                result = move_name_to_perm[move] * result
        return result
    target_perm = apply_move_string_to_perm("f0.f1.f0")
    def apply_move_string_to_string(move_string, string_list):
        # string_list is [A, B, C, D, E, F, G, H, I, J, K, L, M, N] for example
        moves = move_string.split(".")
        result = string_list.copy()
        perm = apply_move_string_to_perm(move_string)
        print(perm)
        return (perm)(string_list)
    print(apply_move_string_to_string("f0.f1.f0", list(range(24))))
    



print(get_target_perm_for_id(0))
selm = "(0 18 23 9)(1 16 22 11)(2 21)(3 20)(4 7)(5 6)(8 19)(10 17)(12 13 15 14)".replace(",", "")
# increment each num in sel by one and within each circle, add a comma
selm = list(map(lambda x: x.replace(")", ""), selm.split("(")))
print(selm)
selm = list(map(lambda x: x.split(" "), selm))
print(selm)
selm = list(map(lambda x: list(map(lambda y: int(y)+1, x)), selm[1:]))
print(selm)
selm = list(map(lambda x: ",".join(map(str, x)), selm))
print(selm)
selm = "(" + ")(".join(selm) + ")"
print(selm)
        

{'f0': Permutation(23)(2, 19, 21, 8)(3, 17, 20, 10)(4, 6, 7, 5), 'f1': Permutation(0, 18, 23, 9)(1, 16, 22, 11)(12, 13, 15, 14), 'r0': Permutation(1, 5, 21, 14)(3, 7, 23, 12)(8, 10, 11, 9), 'r1': Permutation(23)(0, 4, 20, 15)(2, 6, 22, 13)(16, 17, 19, 18), 'd0': Permutation(6, 18, 14, 10)(7, 19, 15, 11)(20, 22, 23, 21), 'd1': Permutation(23)(0, 1, 3, 2)(4, 16, 12, 8)(5, 17, 13, 9)}
(0 18 23 9)(1 16 22 11)(2 21)(3 20)(4 7)(5 6)(8 19)(10 17)(12 13 15 14)
[18, 16, 21, 20, 7, 6, 5, 4, 19, 0, 17, 1, 13, 15, 12, 14, 22, 10, 23, 8, 3, 2, 11, 9]
None
['', '0 18 23 9', '1 16 22 11', '2 21', '3 20', '4 7', '5 6', '8 19', '10 17', '12 13 15 14']
[[''], ['0', '18', '23', '9'], ['1', '16', '22', '11'], ['2', '21'], ['3', '20'], ['4', '7'], ['5', '6'], ['8', '19'], ['10', '17'], ['12', '13', '15', '14']]
[[1, 19, 24, 10], [2, 17, 23, 12], [3, 22], [4, 21], [5, 8], [6, 7], [9, 20], [11, 18], [13, 14, 16, 15]]
['1,19,24,10', '2,17,23,12', '3,22', '4,21', '5,8', '6,7', '9,20', '11,18', '13,14,16,15']
(

In [ ]:
target_df = pd.read_csv("./target.csv")
def get_target_perm_for_id(id):
    perm_list = eval("[" + target_df.iloc[id]["target"] + "]")
    return Permutation(list(map(lambda x: x - 1, perm_list))) ** -1
print(list(get_target_perm_for_id(20)))

def apply_perm_list_to_state_str(state_str, perm):
    letters = state_str.split(";")
    return ";".join([letters[i] for i in perm])
sample_str = "a;b;c;d"
print(apply_perm_list_to_state_str(sample_str, get_target_perm_for_id(20)))

[1, 2, 3, 10, 8, 7, 19, 14, 21, 17, 23, 18, 4, 9, 22, 16, 12, 5, 13, 20, 6, 11, 0, 15]


IndexError: list index out of range

In [ ]:
puzzles.head()

,id,puzzle_type,solution_state,initial_state,num_wildcards
0,0,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;D;A;E;B;A;B;C;A;C;A;D;C;D;F;F;F;E;E;B;F;B;C,0
1,1,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;C;B;B;E;F;A;F;D;B;F;F;E;B;D;A;A;C;D;C;E;A;C,0
2,2,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;F;C;C;F;A;D;D;B;B;A;F;E;B;C;A;A;B;D;F;E;E;C;D,0
3,3,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,A;C;E;C;F;D;E;D;A;A;F;A;B;D;B;F;E;D;B;F;B;C;C;E,0
4,4,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;D;E;D;A;E;F;B;A;C;F;D;F;D;C;A;F;B;C;C;B;E;B;A,0


In [86]:
# get rubix cube initial state for given size n x n times each letter A, B, C, D, E, F
def get_initial_state(n):
    letters = ["A", "B", "C", "D", "E", "F"]
    return ";".join("".join([letter * n**2 for letter in letters]))
print(get_initial_state(4))

A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;A;B;B;B;B;B;B;B;B;B;B;B;B;B;B;B;B;C;C;C;C;C;C;C;C;C;C;C;C;C;C;C;C;D;D;D;D;D;D;D;D;D;D;D;D;D;D;D;D;E;E;E;E;E;E;E;E;E;E;E;E;E;E;E;E;F;F;F;F;F;F;F;F;F;F;F;F;F;F;F;F
